In [5]:
import pandas as pd
import numpy as np


long_fp = "M:/Data Science/Data Analyst/Github & LinkedIN Portfolio Projects/Football Academy Performance Project/football_performance_long1.csv"
summary_fp = "M:/Data Science/Data Analyst/Github & LinkedIN Portfolio Projects/Football Academy Performance Project/football_performance_summary.csv"

df_long = pd.read_csv(long_fp)
df_summary = pd.read_csv(summary_fp)

print("df_long columns:", df_long.columns.tolist())
print("df_summary columns:", df_summary.columns.tolist())

# Define the common column based on your dataframes
# Replace 'player_id' with the actual column name that exists in both dataframes
common_column = 'player_id'  # You need to specify the actual common column name

df = pd.merge(
    df_long,
    df_summary,
    on=common_column,
    how='outer',  # Use 'inner', 'left', 'right', or 'outer' based on your needs
    suffixes=('_long', '_summary')  # Add suffixes to distinguish duplicate column names
)

print("\nCombined DataFrame shape:", df.shape)
print("\nFirst 5 rows of combined DataFrame:")
print(df.head())

print("\nMissing values in combined DataFrame:")
print(df.isnull().sum())

df_long columns: ['match_id', 'match_date', 'player_id', 'player_name', 'age', 'position', 'attendance', 'minutes_played', 'fitness_score', 'stamina', 'speed', 'passing_accuracy', 'tackles', 'goals', 'assists', 'shots_on_target', 'saves', 'yellow_card', 'red_card']
df_summary columns: ['player_id', 'player_name', 'age', 'position', 'matches_played', 'total_minutes', 'avg_fitness', 'avg_stamina', 'avg_speed', 'avg_passing', 'total_goals', 'total_assists', 'total_saves', 'total_tackles', 'yellow_cards', 'red_cards']

Combined DataFrame shape: (960, 34)

First 5 rows of combined DataFrame:
  match_id  match_date player_id player_name_long  age_long position_long  \
0     M200  01-01-2025     P1000            Anvay        12           MID   
1     M201  08-01-2025     P1000            Anvay        12           MID   
2     M202  15-01-2025     P1000            Anvay        12           MID   
3     M203  22-01-2025     P1000            Anvay        12           MID   
4     M204  29-01-202

In [8]:

import re
import pandas as pd
from sqlalchemy import create_engine

bool_indicators = ['is_', 'has_', 'was_', 'did_']
for col in df.columns:
    if any(indicator in col for indicator in bool_indicators):
        try:
            df[col] = df[col].astype(bool)
        except:
            pass


if 'birth_date' in df.columns and pd.api.types.is_datetime64_dtype(df['birth_date']):
    df['age'] = (pd.Timestamp.now() - df['birth_date']).dt.days / 365.25
    df['age_group'] = pd.cut(df['age'], 
                            bins=[0, 12, 15, 18, 21, 25, 30, 100],
                            labels=['U12', 'U15', 'U18', 'U21', '21-25', '26-30', '30+'])


performance_cols = [col for col in numeric_cols if 'performance' in col or 'score' in col or 'rating' in col]
for col in performance_cols:
    if col in df.columns:
        # Min-max normalization to 0-10 scale
        min_val = df[col].min()
        max_val = df[col].max()
        if max_val > min_val:  # Avoid division by zero
            df[f'{col}_normalized'] = 10 * (df[col] - min_val) / (max_val - min_val)


if set(['goals', 'assists', 'passes_completed']).issubset(df.columns):
    df['performance_index'] = (
        df['goals'] * 3 + 
        df['assists'] * 2 + 
        df['passes_completed'] * 0.01
    )

print("\nCleaned DataFrame info:")
print(df.info())
print("\nSample of cleaned data:")
print(df.head())


db_username = 'your_username'
db_password = 'your_password'
db_host = 'localhost'
db_port = '5432'
db_name = 'football_academy_db'
table_name = 'football_performance'


connection_string = f'postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}'
engine = create_engine(connection_string)


try:
    df.to_sql(
        name=table_name,
        con=engine,
        if_exists='replace',  # Options: 'fail', 'replace', 'append'
        index=False,
        chunksize=1000  # Adjust based on your data size
    )
    print(f"\nSuccessfully exported data to PostgreSQL table: {table_name}")
except Exception as e:
    print(f"Error exporting to PostgreSQL: {e}")


Cleaned DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960 entries, 0 to 959
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   match_id                  960 non-null    object 
 1   match_date                960 non-null    object 
 2   player_id                 960 non-null    object 
 3   player_name_long          960 non-null    object 
 4   age_long                  960 non-null    int64  
 5   position_long             960 non-null    object 
 6   attendance                960 non-null    int64  
 7   minutes_played            960 non-null    int64  
 8   fitness_score             960 non-null    float64
 9   stamina                   960 non-null    float64
 10  speed                     960 non-null    float64
 11  passing_accuracy          960 non-null    float64
 12  tackles                   960 non-null    int64  
 13  goals                     960 non-null  

In [10]:
df.describe()

,age_long,attendance,minutes_played,fitness_score,stamina,speed,passing_accuracy,tackles,goals,assists,...,avg_stamina,avg_speed,avg_passing,total_goals,total_assists,total_saves,total_tackles,yellow_cards,red_cards,fitness_score_normalized
count,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000,...,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000
mean,14.791667,0.922917,64.276042,69.838958,60.136875,63.737917,66.767500,1.071875,0.033333,0.034375,...,60.136875,63.737917,66.767500,0.666667,0.687500,2.895833,21.437500,1.416667,0.020833,5.490155
std,1.937053,0.266863,25.100712,9.940482,10.648750,10.460888,12.764259,1.296518,0.185314,0.187919,...,2.621101,2.902787,3.732828,1.143205,1.102774,9.679407,13.874014,1.239485,0.142901,1.603304
min,12.000000,0.000000,0.000000,35.800000,28.800000,32.400000,23.300000,0.000000,0.000000,0.000000,...,52.450000,57.325000,58.055000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13.000000,1.000000,53.000000,63.600000,53.200000,56.675000,58.050000,0.000000,0.000000,0.000000,...,58.895000,61.880000,63.876250,0.000000,0.000000,0.000000,11.750000,0.000000,0.000000,4.483871
50%,15.000000,1.000000,69.500000,70.050000,59.500000,63.900000,67.100000,1.000000,0.000000,0.000000,...,60.052500,63.750000,66.792500,0.000000,0.000000,0.000000,17.000000,1.000000,0.000000,5.524194
75%,16.000000,1.000000,86.000000,76.400000,67.100000,70.625000,75.300000,2.000000,0.000000,0.000000,...,61.923750,65.407500,69.712500,1.000000,1.000000,0.000000,36.000000,2.000000,0.000000,6.548387
max,18.000000,1.000000,90.000000,97.800000,96.900000,94.400000,100.000000,7.000000,2.000000,2.000000,...,64.915000,69.265000,73.390000,4.000000,4.000000,39.000000,45.000000,5.000000,1.000000,10.000000


In [14]:
from sqlalchemy import create_engine

# Step 1: Connect to PostgreSQL
# Replace placeholders with actual details
username = "postgres"      # default user
password = "987021" # the password set during installation
host = "localhost"         # if running locally
port = "5432"              # default PostgreSQL port
database = "Football_Academy"    # the database created in pgAdmin

engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}")

# Step 2: Load DataFrame into PostgreSQL
table_name = "FA_P&E"
df.to_sql(table_name, engine, if_exists="replace", index=False)

print(f"Data successfully loaded into table '{table_name}' in database '{database}'.")

Data successfully loaded into table 'FA_P&E' in database 'Football_Academy'.
